In [5]:
import pandas as pd
import numpy as np

# --- 1. Define a Custom Date Cleaning Function ---
def clean_date_format(date_val):
    """
    Strips special characters and non-digit characters from a date string,
    preparing it for conversion.
    """
    if pd.isna(date_val):
        return np.nan
    # Convert to string and remove all non-digit characters
    cleaned_str = ''.join(filter(str.isdigit, str(date_val)))
    # Ensure it's a plausible length for a date (YYYYMMDD)
    return cleaned_str if len(cleaned_str) == 8 else np.nan

# --- 2. Load and Pre-clean the Dataset ---
checkouts_df = pd.read_csv('data\\checkouts.csv')
print(f"Starting with {len(checkouts_df)} raw records.")

# Apply the custom cleaning function first
checkouts_df['date_checkout'] = checkouts_df['date_checkout'].apply(clean_date_format)
checkouts_df['date_returned'] = checkouts_df['date_returned'].apply(clean_date_format)
print("Pre-cleaned date columns to remove special characters and incorrect formats.")

# --- 3. Master Date Conversion and Validation ---
# Now, convert the clean strings to datetime objects
checkouts_df['date_checkout'] = pd.to_datetime(checkouts_df['date_checkout'], errors='coerce')
checkouts_df['date_returned'] = pd.to_datetime(checkouts_df['date_returned'], errors='coerce')

# Drop any rows where the checkout date is invalid after cleaning
checkouts_df.dropna(subset=['date_checkout'], inplace=True)

# Define the boundaries for our analysis
current_date = pd.Timestamp.now()
earliest_valid_date = pd.Timestamp('2005-01-01')

# Apply the date validation filters
initial_rows = len(checkouts_df)

# RULE 1: Remove future checkout dates
checkouts_df = checkouts_df[checkouts_df['date_checkout'] <= current_date]
print(f"Removed {initial_rows - len(checkouts_df)} records with future checkout dates.")
initial_rows = len(checkouts_df)

# RULE 2: Remove future return dates
checkouts_df = checkouts_df[checkouts_df['date_returned'] <= current_date]
print(f"Removed {initial_rows - len(checkouts_df)} records with future return dates.")
initial_rows = len(checkouts_df)

# RULE 3: Remove checkouts before the start date of 2005
checkouts_df = checkouts_df[checkouts_df['date_checkout'] >= earliest_valid_date]
print(f"Removed {initial_rows - len(checkouts_df)} records with checkout dates before {earliest_valid_date.year}.")
initial_rows = len(checkouts_df)

# Fill missing return dates with the current date
checkouts_df['date_returned'].fillna(current_date, inplace=True)

# RULE 4: Remove records where the return happens before the checkout
checkouts_df = checkouts_df[checkouts_df['date_returned'] >= checkouts_df['date_checkout']]
print(f"Removed {initial_rows - len(checkouts_df)} records where return date was before checkout date.")

print(f"\nEnding with {len(checkouts_df)} valid records to be used for analysis.")
print("-" * 50)


# --- 4. Feature Engineering ---
checkouts_df['loan_duration_days'] = (checkouts_df['date_returned'] - checkouts_df['date_checkout']).dt.days
checkouts_df['status'] = np.where(checkouts_df['loan_duration_days'] > 28, 'Late', 'On Time')

# --- 5. Final Output ---
print("\nSuccessfully processed the data with the corrected rules!")

late_rate = checkouts_df['status'].value_counts(normalize=True).get('Late', 0) * 100
print(f"\nThe late return rate on this fully validated data is: {late_rate:.2f}%")

print("\nPreview of the final, clean DataFrame:")
cleaned_checkouts_file = 'cleaned_checkouts.csv'
checkouts_df.to_csv(cleaned_checkouts_file, index=False)
print(f"Cleaned data has been saved to '{cleaned_checkouts_file}'")
print(checkouts_df.head())

Starting with 2000 raw records.
Pre-cleaned date columns to remove special characters and incorrect formats.
Removed 84 records with future checkout dates.
Removed 153 records with future return dates.
Removed 90 records with checkout dates before 2005.
Removed 157 records where return date was before checkout date.

Ending with 1451 valid records to be used for analysis.
--------------------------------------------------

Successfully processed the data with the corrected rules!

The late return rate on this fully validated data is: 9.10%

Preview of the final, clean DataFrame:
Cleaned data has been saved to 'cleaned_checkouts.csv'
             id                         patron_id           library_id  \
1  HUX-y4oXl04C  8d3f63e1deed89d7ba1bf6a4eb101373  223-222@5xc-jxr-tgk   
2  TQpFnkku2poC  4ae202f8de762591734705e0079d76df  228-222@5xc-jtz-hwk   
3  OQ6sDwAAQBAJ  f9372de3c8ea501601aa3fb59ec0f524  23v-222@5xc-jv7-v4v   
6  CW-7tHAaVR0C  dd9f34e9d65126a2b02003d8ac60aaa4  22c-222@5xc-

C:\Users\jovan\AppData\Local\Temp\ipykernel_2652\245197525.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  checkouts_df['date_returned'].fillna(current_date, inplace=True)


In [11]:
import pandas as pd
import numpy as np

# --- 1. Load the customers dataset ---
customers_df = pd.read_csv('data\\customers.csv')
print(f"--- Starting with {len(customers_df)} records, which will all be preserved. ---")

# --- 2. Define Custom Cleaning Functions ---
def clean_street_address(address):
    """Applies custom capitalization rules to a street address."""
    if pd.isna(address):
        return np.nan
    address = ' '.join(str(address).strip().split())
    processed_words = [word.upper() if len(word) <= 2 else word.capitalize() for word in address.split(' ')]
    return ' '.join(processed_words)

def validate_zipcode(zip_val):
    """Validates ZIP code is 5 digits, else returns 'Unknown'."""
    if pd.isna(zip_val):
        return "Unknown"
    cleaned_zip = ''.join(filter(str.isdigit, str(zip_val).split('.')[0]))
    return cleaned_zip if len(cleaned_zip) == 5 else "Unknown"

# --- 3. Apply Cleaning and Create Validated Columns ---
customers_df['name'] = customers_df['name'].astype(str).str.replace(r'[^a-zA-Z\s]', '', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip().str.title()
customers_df['street_address'] = customers_df['street_address'].apply(clean_street_address)

# UPDATED LOOP: Now includes 'education' and 'occupation'
for col in ['city', 'state', 'education', 'occupation']:
    customers_df[col] = customers_df[col].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip().str.title().replace('Nan', np.nan)
print("Cleaned text columns.")

customers_df['zipcode_validated'] = customers_df['zipcode'].apply(validate_zipcode)
customers_df['birth_date_cleaned'] = pd.to_datetime(customers_df['birth_date'], errors='coerce')
print("Created temporary validated columns for quality check.")

# --- 4. Create the Data Quality Flag Column ---
issues = []
now = pd.Timestamp.now()
earliest_valid_date = pd.Timestamp('1930-01-01')

for index, row in customers_df.iterrows():
    row_issues = []
    if row['zipcode_validated'] == 'Unknown':
        row_issues.append('Invalid ZIP')

    birth_date = row['birth_date_cleaned']
    if pd.isna(birth_date):
        row_issues.append('Missing Birth Date')
    else:
        if birth_date < earliest_valid_date:
            row_issues.append('Implausible Age')
        if birth_date > now:
            row_issues.append('Future Birth Date')

    if not row_issues:
        issues.append('OK')
    else:
        issues.append(', '.join(row_issues))

customers_df['data_quality'] = issues
print("Created 'data_quality' flag column based on all validation rules.")

# --- 5. Finalize the DataFrame ---
customers_df['zipcode'] = customers_df['zipcode_validated']
customers_df = customers_df.drop(columns=['zipcode_validated', 'birth_date_cleaned'])

# --- 6. Final Output ---
print("\n--- Data Quality Report ---")
print(customers_df['data_quality'].value_counts())
print("-" * 50)
print("\n--- Final Data Preview ---")
print(customers_df[['name', 'street_address', 'zipcode', 'birth_date', 'data_quality']].head())

# Save the cleaned and enriched data
customers_df.to_csv('cleaned_customers.csv', index=False)
print("\nSaved the final, clean and enriched customer data to 'cleaned_customers.csv'")

--- Starting with 2000 records, which will all be preserved. ---
Cleaned text columns.
Created temporary validated columns for quality check.
Created 'data_quality' flag column based on all validation rules.

--- Data Quality Report ---
data_quality
OK                                1707
Missing Birth Date                 107
Future Birth Date                   93
Implausible Age                     92
Invalid ZIP, Future Birth Date       1
Name: count, dtype: int64
--------------------------------------------------

--- Final Data Preview ---
                name        street_address zipcode  birth_date  \
0  Cynthia Barnfield       44 NE Meikle PL   97213  2009-09-10   
1    Elizabeth Smith   7511 SE Harrison ST   97215  1956-12-15   
2      Richard Pabla       1404 SE Pine ST   97214  1960-12-18   
3      Charles Baker  12271 N Westshore DR   97217  2105-07-19   
4       Ronald Lydon   5321 NE Skyport Way   97218  1961-03-14   

        data_quality  
0                 OK  
1      

In [7]:
import pandas as pd
import numpy as np

# --- 1. Load the books dataset ---
books_df = pd.read_csv('data\\books.csv')
print(f"--- Starting with {len(books_df)} records ---")

# --- 2. NEW: Clean the 'id' Column ---
# Remove any leading equals signs to prevent Excel formula errors
original_ids = books_df['id'].copy()
books_df['id'] = books_df['id'].str.lstrip('=')
changes = (books_df['id'] != original_ids).sum()
print(f"Cleaned 'id' column: Removed leading '=' from {changes} records.")

# --- 3. Create 'publishedYear' Column ---
year_str = books_df['publishedDate'].astype(str).str.slice(0, 4)
books_df['publishedYear'] = pd.to_numeric(year_str, errors='coerce')
books_df['publishedYear'] = books_df['publishedYear'].astype(pd.Int64Dtype())
print("Created 'publishedYear' column without altering 'publishedDate'.")

# --- 4. Clean 'categories' Column ---
books_df['categories'] = books_df['categories'].astype(str).str.replace(r"\[|\]|'|\"", "", regex=True)
books_df['categories'] = books_df['categories'].replace('nan', np.nan)
print("Cleaned 'categories': Removed brackets, single quotes, and double quotes.")

# --- 5. Verification Step ---
remaining_issues = books_df['categories'].str.contains(r"\[|\]|'|\"", regex=True, na=False)
if remaining_issues.any():
    print("Verification FAILED: Special characters still found in the 'categories' column.")
else:
    print("Verification PASSED: All specified special characters successfully removed from 'categories'.")

# --- 6. Clean 'price' Column ---
books_df['price'] = books_df['price'].astype(str).str.extract(r'(\d+\.?\d*)')
books_df['price'] = pd.to_numeric(books_df['price'], errors='coerce')
print("Cleaned 'price': Extracted numbers and converted to numeric.")

# --- 7. Clean 'pages' Column ---
books_df['pages'] = books_df['pages'].astype(str).str.extract(r'(\d+)')
books_df['pages'] = pd.to_numeric(books_df['pages'], errors='coerce')
books_df['pages'] = books_df['pages'].astype(pd.Int64Dtype())
print("Cleaned 'pages': Extracted numbers and converted to Integer.")

# --- 8. Final Output ---
print("\n--- Final Data Preview ---")
print(books_df[['id', 'publishedDate', 'publishedYear', 'categories', 'price', 'pages']].head())
print("\n--- Final Data Info ---")
books_df.info()

# Save the cleaned data
books_df.to_csv('cleaned_books.csv', index=False)
print("\nSaved the final, clean books data to 'cleaned_books.csv'")

--- Starting with 240 records ---
Cleaned 'id' column: Removed leading '=' from 0 records.
Created 'publishedYear' column without altering 'publishedDate'.
Cleaned 'categories': Removed brackets, single quotes, and double quotes.
Verification PASSED: All specified special characters successfully removed from 'categories'.
Cleaned 'price': Extracted numbers and converted to numeric.
Cleaned 'pages': Extracted numbers and converted to Integer.

--- Final Data Preview ---
             id publishedDate  publishedYear               categories   price  \
0  hVFwAAAAQBAJ    2013-09-11           2013           Social Science   72.99   
1  bRY9AAAAYAAJ          1913           1913              Advertising  469.99   
2  ZapAAAAAIAAJ          1973           1973              Advertising  372.00   
3  A-HthMfF5moC          1894           1894              Advertising  240.99   
4  4Z9JAAAAMAAJ          1944           1944  Government publications  539.00   

   pages  
0    320  
1    654  
2    7

In [8]:
import pandas as pd
import numpy as np

# --- 1. Load the libraries dataset ---
libraries_df = pd.read_csv('data\\libraries.csv')

# --- 2. Define Custom Function for Street Address ---
def clean_street_address(address):
    """
    Applies custom capitalization rules to a street address.
    """
    if pd.isna(address):
        return np.nan
    
    # Collapse multiple spaces and trim
    address = str(address).strip()
    address = ' '.join(address.split())
    
    # Process each word
    processed_words = []
    for word in address.split(' '):
        if len(word) <= 2:
            processed_words.append(word.upper()) # Uppercase for short words (SW, SE)
        else:
            processed_words.append(word.capitalize()) # Capitalize for others
            
    return ' '.join(processed_words)

# Keep a copy of the original postal codes to count changes
original_postal_codes = libraries_df['postal_code'].copy()

# --- 3. Clean 'name' Column ---
libraries_df['name'] = libraries_df['name'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip().str.title()
print("Cleaned 'name': Collapsed spaces and applied title case.")

# --- 4. Clean 'street_address' Column with Custom Function ---
libraries_df['street_address'] = libraries_df['street_address'].apply(clean_street_address)
print("Cleaned 'street_address': Applied custom capitalization.")

# --- 5. Clean 'city' Column ---
libraries_df['city'] = libraries_df['city'].astype(str).str.strip().str.title()
libraries_df['city'] = libraries_df['city'].replace('Nan', np.nan)
print("Cleaned 'city': Trimmed whitespace and capitalized.")

# --- 6. Clean 'region' Column ---
libraries_df['region'] = libraries_df['region'].astype(str).str.strip().str.upper()
libraries_df['region'] = libraries_df['region'].replace('NAN', np.nan)
print("Cleaned 'region': Trimmed whitespace and converted to uppercase.")

# --- 7. Clean 'postal_code' Column ---
def clean_postal_code(code):
    if pd.isna(code):
        return np.nan
    cleaned_code = ''.join(filter(str.isdigit, str(code)))
    return cleaned_code if cleaned_code else np.nan

libraries_df['postal_code'] = libraries_df['postal_code'].apply(clean_postal_code)
changes = (libraries_df['postal_code'] != original_postal_codes).sum()
print(f"Cleaned 'postal_code': Removed special characters. Updated {changes} records.")


# --- 8. Final Output ---
print("\n--- Final Data Preview ---")
print(libraries_df.head())
print("\n--- Final Data Info ---")
libraries_df.info()

# Save the cleaned data
libraries_df.to_csv('cleaned_libraries.csv', index=False)
print("\nSaved the final, clean libraries data to 'cleaned_libraries.csv'")

Cleaned 'name': Collapsed spaces and applied title case.
Cleaned 'street_address': Applied custom capitalization.
Cleaned 'city': Trimmed whitespace and capitalized.
Cleaned 'region': Trimmed whitespace and converted to uppercase.
Cleaned 'postal_code': Removed special characters. Updated 14 records.

--- Final Data Preview ---
                    id                                        name  \
0  226-222@5xc-kc4-fpv       Multnomah County Library Capitol Hill   
1  23v-222@5xc-jv7-v4v          Multnomah County Library Northwest   
2  222-222@5xc-jvf-skf           Multnomah County Library St Johns   
3  227-222@5xc-jww-btv          Multnomah County Library Hillsdale   
4  22d-222@5xc-kcy-8sq  Multnomah County Library Sellwood Moreland   

          street_address      city region postal_code  
0   10723 SW Capitol Hwy  Portland     OR       97219  
1     2300 NW Thurman ST       NaN     OR         NaN  
2  7510 N Charleston Ave  Portland     OR       97203  
3    1525 SW Sunset Blvd 

In [12]:
# --- 2. Handle Empty Strings in Customer Data ---
# Replace empty strings "" with actual null values (NaN) first
for col in ['education', 'occupation', 'gender']:
    customers_df[col] = customers_df[col].replace(r'^\s*$', np.nan, regex=True)
print("Replaced empty strings with null values in preparation for filling.")

# --- 3. Rename Columns for a Clean Merge ---
customers_df.rename(columns={'id': 'patron_id', 'name': 'customer_name', 'street_address': 'customer_address', 'city': 'customer_city'}, inplace=True)
books_df.rename(columns={'id': 'book_id'}, inplace=True)
libraries_df.rename(columns={'id': 'library_id', 'name': 'library_name', 'street_address': 'library_address', 'city': 'library_city'}, inplace=True)
checkouts_df.rename(columns={'id': 'book_id'}, inplace=True)
print("Renamed columns for a clean merge.")

# --- 4. Merge the DataFrames ---
print("Merging data...")
master_df = pd.merge(checkouts_df, customers_df, on='patron_id', how='left')
master_df = pd.merge(master_df, books_df, on='book_id', how='left')
master_df = pd.merge(master_df, libraries_df, on='library_id', how='left')

# --- 5. Fill Remaining Missing Values with "Unknown" ---
# YOUR LOGIC: Fill NaN values in these specific columns
for col in ['education', 'occupation', 'gender']:
    master_df[col].fillna("Unknown", inplace=True)
print("Filled empty education, occupation, and gender fields with 'Unknown'.")


# --- 6. Final Cross-Table Validations ---
master_df['birth_date'] = pd.to_datetime(master_df['birth_date'])
master_df['date_checkout'] = pd.to_datetime(master_df['date_checkout'])

# Validation 1: Patron's age
initial_rows = len(master_df)
master_df = master_df[master_df['date_checkout'] >= master_df['birth_date']].copy()
print(f"Validation 1: Removed {initial_rows - len(master_df)} records where checkout occurred before the patron's birth date.")

# Validation 2: Book's existence
initial_rows = len(master_df)
master_df = master_df[master_df['date_checkout'].dt.year >= master_df['publishedYear']].copy()
print(f"Validation 2: Removed {initial_rows - len(master_df)} records where checkout occurred before the book was published.")

# --- 7. Save the Final Master File ---
master_df.to_csv('master_library_data.csv', index=False)
print("\nSuccess! Your final master data file is ready for Power BI.")
print("Saved to 'master_library_data.csv'")

Replaced empty strings with null values in preparation for filling.
Renamed columns for a clean merge.
Merging data...
Filled empty education, occupation, and gender fields with 'Unknown'.
Validation 1: Removed 143 records where checkout occurred before the patron's birth date.
Validation 2: Removed 53 records where checkout occurred before the book was published.

Success! Your final master data file is ready for Power BI.
Saved to 'master_library_data.csv'


C:\Users\jovan\AppData\Local\Temp\ipykernel_2652\2888261430.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  master_df[col].fillna("Unknown", inplace=True)
